# TensorFlow 패키지 소개

TensorFlow 패키지는 Theano 패키지와 마찬가지로 선형 대수 심볼 컴파일러(Symbolic Linear Algebra Compiler)이다. Theano에 비해 분산처리 기능이 강화되어 있다.

## TensorFlow 기본 사용법

Theano와 유사하게 TensorFlow도 다음과 같은 과정을 거쳐 사용한다.

1. 심볼 변수 정의
2. 심볼 관계 정의
3. 세션 정의
4. 세션 사용

세션(Session)은 Theano의 함수와 유사한 역할을 하며 실제 심볼 변수의 관계를 분석하고 값을 계산해준다.

### 심볼 변수 정의

TensorFlow는 `Variable` 명령으로 정의하며 Theano와 달리 각 심볼 변수의 형태를 스칼라, 벡터, 행렬 등으로 명시적으로 정의하지 않는 대신 초기값을 이용해서 형태를 지정한다. 심볼의 이름은 명시적으로 주지 않아도 된다.

In [2]:
import tensorflow as tf

In [3]:
x = tf.Variable(1.0)
y = tf.Variable(2.0)

In [4]:
type(x), type(y)

(tensorflow.python.ops.variables.Variable,
 tensorflow.python.ops.variables.Variable)

### 심볼 관계 정의

이미 만들어진 심볼 변수에 일반 사칙연산이나 TensorFlow 수학 함수를 사용하여 종속 변수 역할을 하는 심볼 변수를 만들 수 있다.

In [5]:
z = x + y

In [6]:
type(z)

tensorflow.python.framework.ops.Tensor

In [7]:
u = tf.log(z)

In [8]:
type(u)

tensorflow.python.framework.ops.Tensor

### 세션

TensorFlow의 세션은 Theano의 함수와 유사한 역할을 하며 실제로 계산 그래프를 생성하고 값을 계산하기위한 환경을 제공한다. Theano의 함수와 달리 세션 생성과 실행 시작, 종료를 명시해야 한다.

* 세션 생성: `Session` 객체 생성
* 세션 사용: `run` 메서드. 그래프의 값을 실제로 계산한다.
* 세션 종료: `close` 메서드. with 문으로 대체하기도 한다.

In [9]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(z)) 
print(sess.run(u)) 
sess.close()

3.0
1.0986123


### 미분

TensorFlow 도 심볼릭 연산에 의한 미분 계산이 가능하다.

In [10]:
f = x ** 2
fx = tf.gradients(f, [x])

with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(f))    
    print(sess.run(fx))

1.0
[2.0]


### 최적화

TensorFlow에는 Theano와 달리 최적화를 위한 `GradientDescentOptimizer` 등의 클래스가 미리 구현되어 있으므로 사용자가 구현할 필요가 없다.

### tensorboard 용 로그 생성

tensorflow 처리 과정에서 로그 파일을 만들면 추후 tensorboard 등의 프로그램에서 읽어서 웹 화면에 나타낼 뿐이다. 따라서 계산 코드에서 로그 생성을 위한 코드를 추가해 주어야 한다. 

로그 코드는 다음과 같은 순서로 추가한다.

1. `summary.histogram` 명령으로 개별 변수 기록용 summary 생성
2. `summary.merge` 명령으로 개별 변수 기록용 summary를 하나의 summary로 합침
3. 세션 생성후 `tf.summary.FileWriter` 명령으로 기록용 writer 객체 생성(위 예제에서는 `writer`). 이 때 기록할 디렉토리 설정
4. `sess.run` 명령으로 summary 객체(위 예제에서는 `merged`)를 보내면 기록 실시
5. 기록용 write 객체의 `add_summary` 명령으로 summary를 실제 파일에 쓰기.

여기에서는 `/home/dockeruser/logs/`라는 디렉토리에 로그를 저장한다. 실행하기 전에 이 디렉토리에 있던 과거 로그를 삭제한다.

In [11]:
!rm /home/dockeruser/logs/*

## 퍼셉트론

In [12]:
from sklearn.datasets import load_iris
iris = load_iris()

idx = np.in1d(iris.target, [0, 2])
X_data = iris.data[idx, 0:2]
y_data = (iris.target[idx] - 1.0)[:, np.newaxis]

In [13]:
np.random.seed(0)
w = tf.Variable(1e-3 * np.random.randn(2, 1))
b = tf.Variable(1e-3 * np.random.randn(1)[0])
d = tf.tanh(tf.matmul(X_data, w) + b)

zero = np.zeros(1, dtype=np.float64)[0]
cost = tf.reduce_sum(tf.maximum(zero, tf.multiply(-y_data, d)))
optimizer = tf.train.GradientDescentOptimizer(1e-7)
train = optimizer.minimize(cost)

sess = tf.Session()

w_summary = tf.summary.histogram("w", w)
b_summary = tf.summary.histogram("b", b)
d_summary = tf.summary.histogram("d", d)
cost_summary = tf.summary.histogram("cost", cost)
merged = tf.summary.merge([w_summary, b_summary, d_summary, cost_summary])

init = tf.global_variables_initializer()
sess.run(init)

writer = tf.summary.FileWriter("/home/dockeruser/logs", sess.graph)

for i in range(100):
    sess.run(train)
    summary = sess.run(merged)
    writer.add_summary(summary, i)
    if i % 10 == 0:
        print("{:4d}: ".format(i), sess.run(cost))

   0:  0.5494091733308988
  10:  0.4550718147888182
  20:  0.3607278431087564
  30:  0.2663786243415365
  40:  0.17202552496054824
  50:  0.07766991175203047
  60:  0.0009515019028440592
  70:  0.0003168280919044532
  80:  0.0002921058755255818
  90:  0.0002922898747369561


In [14]:
y_pred = sess.run(tf.sign(d))

In [15]:
sess.close()

In [16]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_data, y_pred)

array([[49,  1],
       [ 1, 49]])

## TensorBoard

TensorFlow는 theano와 달리 모형 내부와 결과를 모니터링 할 수 있는 TensorBoard 라는 웹서버를 제공한다.

TensorBoard 웹서버는 포트 6006을 사용하므로 만약 도커를 사용하는 경우에는 다음과 같이 포트를 열고 실행해야 한다.

```
docker run --name rpython -it -p 8888:8888 -p 6006:6006 datascienceschool/rpython
```

### tensorboard 가동

TensorBoard를 가동하기 위해서는 콘솔에서 다음과 같이 로그 디렉토리를 설정하여 실행한다.

```
$ tensorboard --logdir=/home/dockeruser/logs
```

위와 같이 실행한 다음에는 다음 주소로 연결하면 TensorBoard 화면을 볼 수 있다.

http://localhost:6006

또는

http://192.168.99.100:6006  (도커 툴박스를 사용하는 경우)

<img src="https://datascienceschool.net/upfiles/3763526231d843b5a01775a8cae316a4.png" style="width:100%;">

#### 연습 문제 1

위 퍼셉트론 코드를 다음과 같이 고쳐라.

1. y값은 0과 1이라는 값만 가진다.
2. 활성화 함수는 로지스틱 함수를 사용한다.
3. 코스트 함수는 다음과 같은 오차 제곱 합수를 사용한다.
$$ C = \sum_{i=1}^N (y - \hat{y})^2 $$